## The Battle of Neighborhoods

This project is to compare the neighborhoods of the two cities and determine how similar or dissimilar they are.

Import all the libaries required.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

## 1. Download and Explore Dataset

### New York dataset

In [2]:
with open("newyork_data.json") as json_data:
    newyork_data = json.load(json_data)

In [3]:
neighborhoods_data = newyork_data['features']

Tranform the data into a pandas dataframe

In [4]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Examine the resulting dataframe

In [5]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [6]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


Create a map of New York with neighborhoods superimposed on top.

In [7]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### Toronto dataset

Scrape the Wikipedia page https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050 in order to obtain data. Note that this is a previous version of wiki page.

In [10]:
PostalCode = pd.read_html("https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050")[0]
PostalCode.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Ignore cells with a borough that is Not assigned. Note that there is no borough with a Not assigned neighborhood.

In [11]:
CleanData = PostalCode[PostalCode['Borough']!='Not assigned']
CleanData[CleanData['Neighbourhood']=='Not assigned']

,Postcode,Borough,Neighbourhood


In [12]:
CleanData2 = CleanData.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()

Read geographical coordinates. Merge data with geographical coordinates. 

In [13]:
geo_coor=pd.read_csv('Geospatial_Coordinates.csv')
geo_coor.rename(columns={'Postal Code':'Postcode'},inplace=True)
Data_merged = pd.merge(CleanData2, geo_coor, on='Postcode')
Data_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
Toronto_data=Data_merged[Data_merged['Borough'].str.contains("Toronto")]
Toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [15]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Toronto_data['Borough'].unique()),
        Toronto_data.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


In [16]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Borough'], Toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto are 43.653963, -79.387207.


To simplify the analysis (number of functional calls and datasets), Manhattan from New York and Downtown Toronto from Toronto are selected.

In [17]:
Manhattan = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
Manhattan.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [18]:
Toronto_DT = Toronto_data[Toronto_data['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Toronto_DT = Toronto_DT.drop(['Postcode'], axis=1)
Toronto_DT = Toronto_DT.rename(columns={'Neighbourhood':'Neighborhood'})
Toronto_DT.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,Rosedale,43.679563,-79.377529
1,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


Combine data for Manhattan and Downtown Toronto

In [19]:
All_data = Manhattan.append(Toronto_DT)
All_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [20]:
All_data.tail()

,Borough,Neighborhood,Latitude,Longitude
14,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420
15,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846
16,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
17,Downtown Toronto,Christie,43.669542,-79.422564
18,Downtown Toronto,Queen's Park,43.662301,-79.389494


In [21]:
All_data.shape

(59, 4)

### Venue data

In [22]:
CLIENT_ID = 'YOBBOLJTWBIHQ3L3GBDENHHIQTPXZ5TKZRCQRMGLDHGVQQ1J' # your Foursquare ID
CLIENT_SECRET = 'HVTMD0NCB0B0TINMFIEEK5IGVYTUGDQU4UCZOHA4XI3UHHHR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [23]:
# Create a function to pull data for all the neighborhoods 
def getNearbyVenues(bnames, names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for bname, name, lat, lng in zip(bnames,names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            bname,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
Venues_data = getNearbyVenues(bnames=All_data['Borough'],
                                   names=All_data['Neighborhood'],
                                   latitudes=All_data['Latitude'],
                                   longitudes=All_data['Longitude']
                                  )


Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 

Examine the resulting dataframe

In [26]:
Venues_data.head()

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Manhattan,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Manhattan,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Manhattan,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Manhattan,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Manhattan,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


In [27]:
Venues_data.shape

(4631, 8)

In [28]:
Venues_data.groupby('Neighborhood').count()

,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100,100
Battery Park City,92,92,92,92,92,92,92
Berczy Park,56,56,56,56,56,56,56
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16,16
"Cabbagetown, St. James Town",46,46,46,46,46,46,46
Carnegie Hill,100,100,100,100,100,100,100
Central Bay Street,83,83,83,83,83,83,83
Central Harlem,45,45,45,45,45,45,45
Chelsea,100,100,100,100,100,100,100


## 2. Analyze Each Neighborhood

In [29]:
# one hot encoding
Data_onehot = pd.get_dummies(Venues_data[['Venue Category']], prefix="", prefix_sep="")

# add borough and neighborhood column back to dataframe
Data_onehot=Data_onehot.drop(['Neighborhood'], axis=1)
Data_onehot.insert(0,'Neighborhood', Venues_data['Neighborhood'])
Data_onehot.insert(0,'Borough', Venues_data['Borough'])

Data_onehot.head()

,Borough,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photogra

In [30]:
Data_grouped = Data_onehot.groupby(['Borough','Neighborhood']).mean().reset_index()
Data_grouped

,Borough,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photogra

Find the most common venues for each neighborhood

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough','Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Data_grouped['Neighborhood']
neighborhoods_venues_sorted['Borough'] = Data_grouped['Borough']

for ind in np.arange(Data_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 2:] = return_most_common_venues(Data_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Adelaide, King, Richmond",Coffee Shop,Restaurant,Café,Thai Restaurant,Steakhouse,Bar,Hotel,Seafood Restaurant,Lounge,Breakfast Spot
1,Downtown Toronto,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Farmers Market,Café,Cheese Shop,Restaurant,Beer Bar,Seafood Restaurant,Park
2,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Terminal,Airport Lounge,Airport Service,Rental Car Location,Coffee Shop,Boat or Ferry,Boutique,Sculpture Garden,Harbor / Marina,Airport Gate
3,Downtown Toronto,"Cabbagetown, St. James Town",Coffee Shop,Park,Pharmacy,Pizza Place,Pet Store,Bakery,Italian Restaurant,Restaurant,Café,Pub
4,Downtown Toronto,Central Bay Street,Coffee Shop,Italian Restaurant,Ice Cream Shop,Burger Joint,Japanese Restaurant,Sandwich Place,Juice Bar,Department Store,Chinese Restaurant,Restaurant


## 3. Clustering neighborhoods

In [43]:
# set number of clusters
kclusters = 5

grouped_clustering = Data_grouped.drop(['Borough','Neighborhood'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 0, 3, 2, 2, 2])

In [44]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted=neighborhoods_venues_sorted.drop(['Borough'], 1)

Alldata_merged = All_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Alldata_merged = Alldata_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Alldata_merged.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Miscellaneous Shop,Steakhouse,Supplement Shop,Shopping Mall,Donut Shop,Seafood Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,1,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Hotpot Restaurant,Optical Shop,Bakery,Spa,Salon / Barbershop,Dim Sum Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Café,Bakery,Grocery Store,Chinese Restaurant,Deli / Bodega,Mobile Phone Shop,Pizza Place,Donut Shop,Coffee Shop,Spanish Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,0,Mexican Restaurant,Lounge,Pizza Place,Restaurant,Café,Chinese Restaurant,Deli / Bodega,American Restaurant,Park,Frozen Yogurt Shop
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Pizza Place,Coffee Shop,Deli / Bodega,Café,Mexican Restaurant,Yoga Studio,Bakery,School,Sandwich Place,Caribbean Restaurant


Display each cluster 

In [35]:
Alldata_merged.loc[Alldata_merged['Cluster Labels'] == 0, Alldata_merged.columns[[0] + [1] + list(range(5, Alldata_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Manhattan,Washington Heights,Café,Bakery,Grocery Store,Chinese Restaurant,Deli / Bodega,Mobile Phone Shop,Pizza Place,Donut Shop,Coffee Shop,Spanish Restaurant
3,Manhattan,Inwood,Mexican Restaurant,Lounge,Pizza Place,Restaurant,Café,Chinese Restaurant,Deli / Bodega,American Restaurant,Park,Frozen Yogurt Shop
4,Manhattan,Hamilton Heights,Pizza Place,Coffee Shop,Deli / Bodega,Café,Mexican Restaurant,Yoga Studio,Bakery,School,Sandwich Place,Caribbean Restaurant
7,Manhattan,East Harlem,Mexican Restaurant,Bakery,Thai Restaurant,Deli / Bodega,Latin American Restaurant,Pizza Place,Park,Spa,Convenience Store,Pharmacy
20,Manhattan,Lower East Side,Chinese Restaurant,Coffee Shop,Café,Art Gallery,Pizza Place,Sandwich Place,Japanese Restaurant,Bakery,Cocktail Bar,Ramen Restaurant
22,Manhattan,Little Italy,Café,Bakery,Bubble Tea Shop,Sandwich Place,Salon / Barbershop,Mediterranean Restaurant,Cocktail Bar,Italian Restaurant,Yoga Studio,Hotel
25,Manhattan,Manhattan Valley,Pizza Place,Bar,Coffee Shop,Indian Restaurant,Yoga Studio,Mexican Restaurant,Playground,Thai Restaurant,Korean Restaurant,Szechuan Restaurant
36,Manhattan,Tudor City,Café,Park,Mexican Restaurant,Coffee Shop,Pizza Place,Deli / Bodega,Diner,Thai Restaurant,Garden,Dog Run
12,Downtown Toronto,"Harbord, University of Toronto",Café,Bakery,Japanese Restaurant,Italian Restaurant,Restaurant,Bookstore,Bar,Bank,Beer Store,Gym
13,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",Bar,Vietnamese Restaurant,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Bakery,Mexican Restaurant,Dumpling Restaurant,Comfort Food Restaurant,Farmers Market


In [36]:
Alldata_merged.loc[Alldata_merged['Cluster Labels'] == 1, Alldata_merged.columns[[0] + [1] + list(range(5, Alldata_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Miscellaneous Shop,Steakhouse,Supplement Shop,Shopping Mall,Donut Shop,Seafood Restaurant
1,Manhattan,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Hotpot Restaurant,Optical Shop,Bakery,Spa,Salon / Barbershop,Dim Sum Restaurant
5,Manhattan,Manhattanville,Coffee Shop,Seafood Restaurant,Park,Mexican Restaurant,Italian Restaurant,Spanish Restaurant,Falafel Restaurant,Gastropub,Sushi Restaurant,Lounge
6,Manhattan,Central Harlem,African Restaurant,American Restaurant,Pizza Place,Bar,Seafood Restaurant,French Restaurant,Chinese Restaurant,Market,Salon / Barbershop,Tapas Restaurant
8,Manhattan,Upper East Side,Coffee Shop,Italian Restaurant,Exhibit,Bakery,Art Gallery,Gym / Fitness Center,Juice Bar,Pizza Place,Hotel,French Restaurant
9,Manhattan,Yorkville,Italian Restaurant,Gym,Coffee Shop,Bar,Deli / Bodega,Pizza Place,Sushi Restaurant,Diner,Japanese Restaurant,Mexican Restaurant
10,Manhattan,Lenox Hill,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Cocktail Bar,Café,Gym / Fitness Center,Gym,Burger Joint,Sporting Goods Shop
11,Manhattan,Roosevelt Island,Coffee Shop,Sandwich Place,Japanese Restaurant,Athletics & Sports,Park,Residential Building (Apartment / Condo),Deli / Bodega,Bubble Tea Shop,Kosher Restaurant,Pizza Place
12,Manhattan,Upper West Side,Italian Restaurant,Wine Bar,Bar,Café,Coffee Shop,Mediterranean Restaurant,Indian Restaurant,Bakery,Dessert Shop,Ice Cream Shop
13,Manhattan,Lincoln Square,Theater,Plaza,Café,Italian Restaurant,Performing Arts Venue,Concert Hall,French Restaurant,American Restaurant,Gym / Fitness Center,Indie Movie Theater


In [37]:
Alldata_merged.loc[Alldata_merged['Cluster Labels'] == 2, Alldata_merged.columns[[0] + [1] + list(range(5, Alldata_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,"Cabbagetown, St. James Town",Coffee Shop,Park,Pharmacy,Pizza Place,Pet Store,Bakery,Italian Restaurant,Restaurant,Café,Pub
2,Downtown Toronto,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Café,Men's Store,Hotel,Bubble Tea Shop,Mediterranean Restaurant
3,Downtown Toronto,Harbourfront,Coffee Shop,Bakery,Pub,Park,Theater,Café,Mexican Restaurant,Restaurant,Distribution Center,Chocolate Shop
4,Downtown Toronto,"Ryerson, Garden District",Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Pizza Place,Theater,Bubble Tea Shop,Electronics Store
5,Downtown Toronto,St. James Town,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Breakfast Spot,Bakery,Clothing Store,Beer Bar,Diner
6,Downtown Toronto,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Farmers Market,Café,Cheese Shop,Restaurant,Beer Bar,Seafood Restaurant,Park
7,Downtown Toronto,Central Bay Street,Coffee Shop,Italian Restaurant,Ice Cream Shop,Burger Joint,Japanese Restaurant,Sandwich Place,Juice Bar,Department Store,Chinese Restaurant,Restaurant
8,Downtown Toronto,"Adelaide, King, Richmond",Coffee Shop,Restaurant,Café,Thai Restaurant,Steakhouse,Bar,Hotel,Seafood Restaurant,Lounge,Breakfast Spot
9,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Fried Chicken Joint,Scenic Lookout,Brewery,Restaurant,Sporting Goods Shop
10,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Japanese Restaurant,Gastropub,Bar,Seafood Restaurant,American Restaurant


In [38]:
Alldata_merged.loc[Alldata_merged['Cluster Labels'] == 3, Alldata_merged.columns[[0] + [1] + list(range(5, Alldata_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,Christie,Grocery Store,Café,Park,Candy Store,Coffee Shop,Baby Store,Nightclub,Athletics & Sports,Italian Restaurant,Diner


In [39]:
Alldata_merged.loc[Alldata_merged['Cluster Labels'] == 4, Alldata_merged.columns[[0] + [1] + list(range(5, Alldata_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Rosedale,Park,Trail,Playground,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner


In [49]:
# set number of clusters
kclusters = 10

grouped_clustering2 = Data_grouped.drop(['Borough','Neighborhood'], 1)

# run k-means clustering
kmeans2 = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering2)

# add clustering labels
neighborhoods_venues_sorted=neighborhoods_venues_sorted.drop(['Cluster Labels'], 1)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans2.labels_)

Alldata_merged2 = All_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Alldata_merged2 = Alldata_merged2.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Alldata_merged2.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,7,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Miscellaneous Shop,Steakhouse,Supplement Shop,Shopping Mall,Donut Shop,Seafood Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,1,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Hotpot Restaurant,Optical Shop,Bakery,Spa,Salon / Barbershop,Dim Sum Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,1,Café,Bakery,Grocery Store,Chinese Restaurant,Deli / Bodega,Mobile Phone Shop,Pizza Place,Donut Shop,Coffee Shop,Spanish Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,4,Mexican Restaurant,Lounge,Pizza Place,Restaurant,Café,Chinese Restaurant,Deli / Bodega,American Restaurant,Park,Frozen Yogurt Shop
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4,Pizza Place,Coffee Shop,Deli / Bodega,Café,Mexican Restaurant,Yoga Studio,Bakery,School,Sandwich Place,Caribbean Restaurant


Display each cluster 

In [51]:
Alldata_merged2.loc[Alldata_merged2['Cluster Labels'] == 0, Alldata_merged2.columns[[0] + [1] + list(range(5, Alldata_merged2.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Manhattan,Manhattanville,Coffee Shop,Seafood Restaurant,Park,Mexican Restaurant,Italian Restaurant,Spanish Restaurant,Falafel Restaurant,Gastropub,Sushi Restaurant,Lounge
18,Downtown Toronto,Queen's Park,Coffee Shop,Park,Yoga Studio,Sandwich Place,Bar,Sushi Restaurant,Beer Bar,Fried Chicken Joint,Mexican Restaurant,Burger Joint


In [52]:
Alldata_merged2.loc[Alldata_merged2['Cluster Labels'] == 1, Alldata_merged2.columns[[0] + [1] + list(range(5, Alldata_merged2.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Manhattan,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Hotpot Restaurant,Optical Shop,Bakery,Spa,Salon / Barbershop,Dim Sum Restaurant
2,Manhattan,Washington Heights,Café,Bakery,Grocery Store,Chinese Restaurant,Deli / Bodega,Mobile Phone Shop,Pizza Place,Donut Shop,Coffee Shop,Spanish Restaurant
12,Manhattan,Upper West Side,Italian Restaurant,Wine Bar,Bar,Café,Coffee Shop,Mediterranean Restaurant,Indian Restaurant,Bakery,Dessert Shop,Ice Cream Shop
13,Manhattan,Lincoln Square,Theater,Plaza,Café,Italian Restaurant,Performing Arts Venue,Concert Hall,French Restaurant,American Restaurant,Gym / Fitness Center,Indie Movie Theater
15,Manhattan,Midtown,Hotel,Coffee Shop,Clothing Store,Theater,Steakhouse,French Restaurant,Bookstore,Sporting Goods Shop,Cocktail Bar,Bakery
17,Manhattan,Chelsea,Coffee Shop,Bakery,Italian Restaurant,Ice Cream Shop,Wine Shop,American Restaurant,Hotel,Tapas Restaurant,Cocktail Bar,Theater
18,Manhattan,Greenwich Village,Italian Restaurant,Clothing Store,Sushi Restaurant,Café,French Restaurant,Indian Restaurant,Cosmetics Shop,Cocktail Bar,Boutique,Dessert Shop
20,Manhattan,Lower East Side,Chinese Restaurant,Coffee Shop,Café,Art Gallery,Pizza Place,Sandwich Place,Japanese Restaurant,Bakery,Cocktail Bar,Ramen Restaurant
22,Manhattan,Little Italy,Café,Bakery,Bubble Tea Shop,Sandwich Place,Salon / Barbershop,Mediterranean Restaurant,Cocktail Bar,Italian Restaurant,Yoga Studio,Hotel
23,Manhattan,Soho,Clothing Store,Women's Store,Boutique,Shoe Store,Men's Store,Art Gallery,Bakery,Sporting Goods Shop,Mediterranean Restaurant,Seafood Restaurant


In [53]:
Alldata_merged2.loc[Alldata_merged2['Cluster Labels'] == 2, Alldata_merged2.columns[[0] + [1] + list(range(5, Alldata_merged2.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Rosedale,Park,Trail,Playground,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner


In [54]:
Alldata_merged2.loc[Alldata_merged2['Cluster Labels'] == 3, Alldata_merged2.columns[[0] + [1] + list(range(5, Alldata_merged2.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Café,Men's Store,Hotel,Bubble Tea Shop,Mediterranean Restaurant
3,Downtown Toronto,Harbourfront,Coffee Shop,Bakery,Pub,Park,Theater,Café,Mexican Restaurant,Restaurant,Distribution Center,Chocolate Shop
4,Downtown Toronto,"Ryerson, Garden District",Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Pizza Place,Theater,Bubble Tea Shop,Electronics Store
5,Downtown Toronto,St. James Town,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Breakfast Spot,Bakery,Clothing Store,Beer Bar,Diner
6,Downtown Toronto,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Farmers Market,Café,Cheese Shop,Restaurant,Beer Bar,Seafood Restaurant,Park
7,Downtown Toronto,Central Bay Street,Coffee Shop,Italian Restaurant,Ice Cream Shop,Burger Joint,Japanese Restaurant,Sandwich Place,Juice Bar,Department Store,Chinese Restaurant,Restaurant
8,Downtown Toronto,"Adelaide, King, Richmond",Coffee Shop,Restaurant,Café,Thai Restaurant,Steakhouse,Bar,Hotel,Seafood Restaurant,Lounge,Breakfast Spot
9,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Fried Chicken Joint,Scenic Lookout,Brewery,Restaurant,Sporting Goods Shop
10,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Japanese Restaurant,Gastropub,Bar,Seafood Restaurant,American Restaurant
11,Downtown Toronto,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Japanese Restaurant,Deli / Bodega,Italian Restaurant,Seafood Restaurant


In [55]:
Alldata_merged2.loc[Alldata_merged2['Cluster Labels'] == 4, Alldata_merged2.columns[[0] + [1] + list(range(5, Alldata_merged2.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Manhattan,Inwood,Mexican Restaurant,Lounge,Pizza Place,Restaurant,Café,Chinese Restaurant,Deli / Bodega,American Restaurant,Park,Frozen Yogurt Shop
4,Manhattan,Hamilton Heights,Pizza Place,Coffee Shop,Deli / Bodega,Café,Mexican Restaurant,Yoga Studio,Bakery,School,Sandwich Place,Caribbean Restaurant
7,Manhattan,East Harlem,Mexican Restaurant,Bakery,Thai Restaurant,Deli / Bodega,Latin American Restaurant,Pizza Place,Park,Spa,Convenience Store,Pharmacy
9,Manhattan,Yorkville,Italian Restaurant,Gym,Coffee Shop,Bar,Deli / Bodega,Pizza Place,Sushi Restaurant,Diner,Japanese Restaurant,Mexican Restaurant
10,Manhattan,Lenox Hill,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Cocktail Bar,Café,Gym / Fitness Center,Gym,Burger Joint,Sporting Goods Shop
11,Manhattan,Roosevelt Island,Coffee Shop,Sandwich Place,Japanese Restaurant,Athletics & Sports,Park,Residential Building (Apartment / Condo),Deli / Bodega,Bubble Tea Shop,Kosher Restaurant,Pizza Place
19,Manhattan,East Village,Bar,Ice Cream Shop,Wine Bar,Pizza Place,Cocktail Bar,Mexican Restaurant,Ramen Restaurant,Korean Restaurant,Chinese Restaurant,Italian Restaurant
25,Manhattan,Manhattan Valley,Pizza Place,Bar,Coffee Shop,Indian Restaurant,Yoga Studio,Mexican Restaurant,Playground,Thai Restaurant,Korean Restaurant,Szechuan Restaurant
27,Manhattan,Gramercy,Bar,Italian Restaurant,Pizza Place,Mexican Restaurant,Bagel Shop,American Restaurant,Grocery Store,Spa,Hotel,Ice Cream Shop
35,Manhattan,Turtle Bay,Italian Restaurant,Steakhouse,Sushi Restaurant,Coffee Shop,Wine Bar,Ramen Restaurant,Indian Restaurant,Hotel,Café,French Restaurant


In [56]:
Alldata_merged2.loc[Alldata_merged2['Cluster Labels'] == 5, Alldata_merged2.columns[[0] + [1] + list(range(5, Alldata_merged2.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,Christie,Grocery Store,Café,Park,Candy Store,Coffee Shop,Baby Store,Nightclub,Athletics & Sports,Italian Restaurant,Diner


In [57]:
Alldata_merged2.loc[Alldata_merged2['Cluster Labels'] == 6, Alldata_merged2.columns[[0] + [1] + list(range(5, Alldata_merged2.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Manhattan,Central Harlem,African Restaurant,American Restaurant,Pizza Place,Bar,Seafood Restaurant,French Restaurant,Chinese Restaurant,Market,Salon / Barbershop,Tapas Restaurant
8,Manhattan,Upper East Side,Coffee Shop,Italian Restaurant,Exhibit,Bakery,Art Gallery,Gym / Fitness Center,Juice Bar,Pizza Place,Hotel,French Restaurant
14,Manhattan,Clinton,Theater,Italian Restaurant,Gym / Fitness Center,Coffee Shop,American Restaurant,Wine Shop,Spa,Sandwich Place,Hotel,Gym
16,Manhattan,Murray Hill,Sandwich Place,Coffee Shop,Japanese Restaurant,American Restaurant,Gym / Fitness Center,Gym,Italian Restaurant,Hotel,Mediterranean Restaurant,Cocktail Bar
21,Manhattan,Tribeca,American Restaurant,Park,Italian Restaurant,Spa,Café,Greek Restaurant,Coffee Shop,Wine Bar,Wine Shop,Poke Place
26,Manhattan,Morningside Heights,American Restaurant,Bookstore,Park,Coffee Shop,Burger Joint,Deli / Bodega,Sandwich Place,Spa,Food Truck,Outdoor Sculpture
28,Manhattan,Battery Park City,Park,Coffee Shop,Hotel,Shopping Mall,Women's Store,Wine Shop,Clothing Store,Memorial Site,Gym,Gourmet Shop
29,Manhattan,Financial District,Coffee Shop,Bar,American Restaurant,Pizza Place,Gym,Italian Restaurant,Hotel,Steakhouse,Wine Shop,Gym / Fitness Center
32,Manhattan,Civic Center,Coffee Shop,Gym / Fitness Center,French Restaurant,Hotel,Yoga Studio,Park,Spa,Bakery,Sandwich Place,Cocktail Bar
34,Manhattan,Sutton Place,Gym / Fitness Center,Italian Restaurant,Furniture / Home Store,Gym,Coffee Shop,Indian Restaurant,Yoga Studio,Beer Garden,Beer Bar,Sushi Restaurant


In [58]:
Alldata_merged2.loc[Alldata_merged2['Cluster Labels'] == 7, Alldata_merged2.columns[[0] + [1] + list(range(5, Alldata_merged2.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Miscellaneous Shop,Steakhouse,Supplement Shop,Shopping Mall,Donut Shop,Seafood Restaurant


In [59]:
Alldata_merged2.loc[Alldata_merged2['Cluster Labels'] == 8, Alldata_merged2.columns[[0] + [1] + list(range(5, Alldata_merged2.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Manhattan,Stuyvesant Town,Boat or Ferry,Park,Bar,Pet Service,Heliport,Gas Station,Gym / Fitness Center,Cocktail Bar,Coffee Shop,Harbor / Marina


In [60]:
Alldata_merged2.loc[Alldata_merged2['Cluster Labels'] == 9, Alldata_merged2.columns[[0] + [1] + list(range(5, Alldata_merged2.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Terminal,Airport Lounge,Airport Service,Rental Car Location,Coffee Shop,Boat or Ferry,Boutique,Sculpture Garden,Harbor / Marina,Airport Gate
